# SMPT: Experimental Results

In this notebook, we report on some experimental results obtained with SMPT, our prototype implementation of an SMT-based model-checker for Petri nets with reduction equations.

### Setup Analysis

Import librairies

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns

Path to data

In [ ]:
path_data = 'OUTPUTS/merged/'

Settings

In [ ]:
TIMEOUT = 60

## Load Data

Data frame corresponding to the reduction data.

In [ ]:
# Read 'reduction.csv'
df_reduction = pd.read_csv(path_data + 'reduction.csv')
df_reduction

Data frame corresponding to the ReachabilityCardinality properties.

In [ ]:
# Read 'RC.csv'
df_RC = pd.read_csv(path_data + 'RC.csv')
df_RC

Data frame corresponding to the ReachabilityFireability properties.

In [ ]:
# Read 'RF.csv'
df_RF = pd.read_csv(path_data + 'RF.csv')
df_RF

Data frame corresponding to the ReachabilityDeadlock properties.

In [ ]:
# Read 'RD.csv'
df_RD = pd.read_csv(path_data + 'RD.csv')
df_RD

Concatenate properties data frames into one. 

In [ ]:
# Concatenate 'ReachabilityCardinality', ''ReachabilityFireability' and 'ReachabilityDeadlock' 
df_properties = pd.concat([df_RC, df_RF, df_RD])

# Convert correctness to str
df_properties = df_properties.astype({'CORRECTNESS_WITH_REDUCTION': 'str', 'CORRECTNESS_WITHOUT_REDUCTION': 'str'})
df_properties

## Tool Confidence rate

### Reliability

Reliability with reduction.

In [ ]:
reliability_with_reduction = df_properties.query('CORRECTNESS_WITH_REDUCTION == "False"').shape[0] / df_properties.shape[0] * 100
reliability_with_reduction

Reliability without reduction.

In [ ]:
reliability_without_reduction = df_properties.query('CORRECTNESS_WITHOUT_REDUCTION == "False"').shape[0] / df_properties.shape[0] * 100
reliability_without_reduction

### Correct Values

Correct values with reduction.

In [ ]:
correct_values_with_reduction = df_properties.query('CORRECTNESS_WITH_REDUCTION == "True"').shape[0]
correct_values_with_reduction

Correct values without reduction.

In [ ]:
correct_values_without_reduction = df_properties.query('CORRECTNESS_WITHOUT_REDUCTION == "True"').shape[0]
correct_values_without_reduction

### Summary Table

In [ ]:
summary = {'Reliability': [reliability_with_reduction, reliability_without_reduction], 'Correct Values': [correct_values_with_reduction, correct_values_without_reduction]}
pd.DataFrame(data=summary, index=['With reduction', 'Without reduction'])

## Analysis

Performance evalutation of the *polyhedral abstraction* approach.

### Reduction Ratio 

Reduction ratio among instances.

In [ ]:
# Count instances with same reduction ratio
ratio_frequency = df_reduction['RATIO'].value_counts().sort_index(ascending=False)

# Cumulative data frame
df_ratio_frequency = pd.DataFrame({'INSTANCES': ratio_frequency.values}).cumsum()
df_ratio_frequency['RATIO'] = ratio_frequency.index

# Add row '0'
df_first_row = pd.DataFrame([{'INSTANCES': 1, 'RATIO': df_ratio_frequency['RATIO'][0]}])
df_ratio_frequency = pd.concat([df_first_row, df_ratio_frequency])

# Draw instances reduction ratio
df_ratio_frequency.plot.area(x='INSTANCES', ylim=(0,100), color='cornflowerblue', figsize=(13,3), legend='')
plt.xlabel('Number of instances', fontsize=13)
plt.ylabel('Reduction ratio (%)', fontsize=13)

plt.show()

### General Performance Overview

In [ ]:
# Get properties with reduction ratio
df_properties_with_ratio = df_properties.join(df_reduction.set_index('INSTANCE'), on='INSTANCE')

# Display mean reduction time
print('Mean reduction time: ', df_properties_with_ratio['TIME'].mean())

# Display monotonic properties rate
print('Monotonic properties rate:', df_properties_with_ratio.query('MONOTONIC == True').shape[0] / df_properties_with_ratio.shape[0] * 100)

In [ ]:
def information_per_reduction_range(ratio_min, ratio_max):
    """ Return information for a given reduction range.
    """
    df = df_properties_with_ratio.query('RATIO >= {} and RATIO <= {}'.format(ratio_min, ratio_max))
    
    reduction_range = '{}-{}%'.format(ratio_min, ratio_max)
    
    number_instances = df[['INSTANCE']].drop_duplicates().shape[0]
    
    number_properties = df.shape[0]
    number_monotonic_properties = df.query('MONOTONIC == True').shape[0]
    
    bmc_with_reduction = df.query('METHOD_WITH_REDUCTION == "BMC"').shape[0]
    ic3_with_reduction = df.query('METHOD_WITH_REDUCTION == "IC3"').shape[0]
    total_with_reduction = bmc_with_reduction + ic3_with_reduction
    
    bmc_without_reduction = df.query('METHOD_WITHOUT_REDUCTION == "BMC"').shape[0]
    ic3_without_reduction = df.query('METHOD_WITHOUT_REDUCTION == "IC3"').shape[0]
    total_without_reduction = bmc_without_reduction + ic3_without_reduction
    
    gain = (1 - total_without_reduction / total_with_reduction) * 100
    
    return [reduction_range, number_instances, number_properties, number_monotonic_properties, bmc_with_reduction, ic3_with_reduction, total_with_reduction, bmc_without_reduction, ic3_without_reduction, total_without_reduction, gain]

Summary table.

In [ ]:
performance_overview = pd.DataFrame([information_per_reduction_range(ratio_min, ratio_max) for ratio_min, ratio_max in [[30, 100], [30, 70], [60, 99], [100,100]]], columns=['Reduction Ratio', 'Number of Instances', 'Number of Properties Examinated', 'Number of Monotonic Properties', 'BMC With Reduction', 'IC3 With Reduction', 'Total With Reduction', 'BMC Without Reduction', 'IC3 Without Reduction', 'Total Without Reduction', 'Gain (%)'])
performance_overview.set_index('Reduction Ratio')

### Property Computation Times: With VS Without Reduction

Comparaison of the number of processed properties in a limited time between with and without reduction.

In [ ]:
def time_with_vs_without_reduction(ratio_min, ratio_max):
    """ Plot property computation times with vs without reduction.
    """
    # Get computed properties for a given reduction range
    df = df_properties.join(df_reduction.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and (TIME_WITH_REDUCTION == TIME_WITH_REDUCTION or TIME_WITHOUT_REDUCTION == TIME_WITHOUT_REDUCTION)'.format(ratio_min, ratio_max)).round({'TIME_WITH_REDUCTION': 1, 'TIME_WITHOUT_REDUCTION': 1})

    # Replace timeout NaN values by the timeout value
    df.loc[df.TIME_WITH_REDUCTION != df.TIME_WITH_REDUCTION, 'TIME_WITH_REDUCTION'] = TIMEOUT
    df.loc[df.TIME_WITHOUT_REDUCTION != df.TIME_WITHOUT_REDUCTION, 'TIME_WITHOUT_REDUCTION'] = TIMEOUT

    x = df['TIME_WITHOUT_REDUCTION'].to_numpy()
    y = df['TIME_WITH_REDUCTION'].to_numpy()

    ax = sns.jointplot(x=x, y=y, marginal_kws=dict(bins=24))
    ax.ax_joint.cla()
    plt.sca(ax.ax_joint)
   
    plt.hist2d(x, y, 24, norm=mpl.colors.LogNorm(vmin=0.2), cmap=plt.cm.Blues)
    
    plt.xlabel('Computation time without reduction (s)', fontsize=13)
    plt.ylabel('Computation time with reduction (s)', fontsize=13)
    
    plt.plot([0, TIMEOUT], [0, TIMEOUT], linewidth=1.5, linestyle='--', color='black', scalex=False, scaley=False)

    cbar = ax.fig.add_axes([1, 0.1, .03, .7])
    cb = plt.colorbar(cax=cbar)

    plt.show()

Reduction range: [30, 70].

In [ ]:
time_with_vs_without_reduction(30, 70)

Reduction range: [60, 100[.

In [ ]:
time_with_vs_without_reduction(60, 99)